<a href="https://colab.research.google.com/github/ekbhatt25/crop-disease-detection/blob/main/Crop_Disease_Detection_CNN_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # mount Google Drive to save model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import libraries for confusion matrices
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
# Load dataset
(dataset, info) = tfds.load('plant_village', with_info=True, as_supervised=True)
class_names = info.features['label'].names
print(f"Loaded dataset with {len(class_names)} classes: {class_names}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/plant_village/incomplete.6K31WY_1.0.2/plant_village-train.tfrecord*...:   …

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.
Loaded dataset with 38 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', '

In [ ]:
# Preprocessing
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
# Train/test split #
total_examples = info.splits['train'].num_examples
train_size = int(0.8 * total_examples)

train_ds = dataset['train'].take(train_size).map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = dataset['train'].skip(train_size).map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Build model
from tensorflow.keras import layers, models

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Checkpoint
checkpoint_path = '/content/drive/MyDrive/plant_village_best_model.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max')

In [ ]:
# Train
history = model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[checkpoint])

Epoch 1/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5672 - loss: 1.5934

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 85s 58ms/step - accuracy: 0.5673 - loss: 1.5929 - val_accuracy: 0.8484 - val_loss: 0.4621
Epoch 2/10
1356/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8662 - loss: 0.4203

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 75s 54ms/step - accuracy: 0.8662 - loss: 0.4201 - val_accuracy: 0.8890 - val_loss: 0.3629
Epoch 3/10
1355/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9236 - loss: 0.2333

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 55s 40ms/step - accuracy: 0.9236 - loss: 0.2332 - val_accuracy: 0.9113 - val_loss: 0.2900
Epoch 4/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 98s 52ms/step - accuracy: 0.9502 - loss: 0.1453 - val_accuracy: 0.9000 - val_loss: 0.3572
Epoch 5/10
1357/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9621 - loss: 0.1066

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 73s 53ms/step - accuracy: 0.9621 - loss: 0.1066 - val_accuracy: 0.9196 - val_loss: 0.2965
Epoch 6/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 73s 53ms/step - accuracy: 0.9706 - loss: 0.0829 - val_accuracy: 0.9009 - val_loss: 0.3909
Epoch 7/10
1355/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9744 - loss: 0.0799

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 55s 40ms/step - accuracy: 0.9744 - loss: 0.0798 - val_accuracy: 0.9230 - val_loss: 0.3318
Epoch 8/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 74s 54ms/step - accuracy: 0.9789 - loss: 0.0615 - val_accuracy: 0.9172 - val_loss: 0.3564
Epoch 9/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9808 - loss: 0.0579

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 73s 53ms/step - accuracy: 0.9808 - loss: 0.0579 - val_accuracy: 0.9264 - val_loss: 0.3332
Epoch 10/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 62s 38ms/step - accuracy: 0.9860 - loss: 0.0432 - val_accuracy: 0.9248 - val_loss: 0.3203


In [ ]:
# Evaluate
loss, accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {accuracy:.4f}, Test loss: {loss:.4f}")

340/340 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.9274 - loss: 0.3121
Test accuracy: 0.9248, Test loss: 0.3203


In [ ]:
# Get true labels and predictions
y_true = []
y_pred = []

for images, labels in test_ds.unbatch():
    y_true.append(labels.numpy())
    preds = model.predict(tf.expand_dims(images, axis=0))
    y_pred.append(np.argmax(preds))

Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

KeyboardInterrupt: 

In [ ]:
cm = confusion_matrix(y_true, y_pred)

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=class_names))